In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("SparkSession") \
     .getOrCreate()

sc = spark.sparkContext 
sc

<SparkContext master=yarn appName=SparkSession>

In [7]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import DateType
import pyspark.sql.functions as func

In [8]:
schema = StructType([
    StructField("stream_ID", StringType(), True),
    StructField("current_views", StringType(), True),
    StructField("stream_created_time", DateType(), True),
    StructField("game_name", StringType(), True),
    StructField("broadcaster_ID", StringType(), True),
    StructField("broadcaster_name", StringType(), True),
    StructField("delay_setting", StringType(), True),
    StructField("follower_number", StringType(), True),
    StructField("partner_status", StringType(), True),
    StructField("broadcaster_language", StringType(), True),
    StructField("total_views_of_this_broadcaster", StringType(), True),
    StructField("language", StringType(), True),
    StructField("broadcasters_created_time", StringType(), True),
    StructField("playback_bitrate", StringType(), True),
    StructField("source_resolution", StringType(), True),
])

In [9]:
twitch = spark.read.csv('s3://502-project/Twitch_data',sep = "\t", schema=schema)
twitch.show(10)

+-----------+-------------+-------------------+--------------------+--------------+-------------------+-------------+---------------+--------------+--------------------+-------------------------------+--------+-------------------------+----------------+-----------------+
|  stream_ID|current_views|stream_created_time|           game_name|broadcaster_ID|   broadcaster_name|delay_setting|follower_number|partner_status|broadcaster_language|total_views_of_this_broadcaster|language|broadcasters_created_time|playback_bitrate|source_resolution|
+-----------+-------------+-------------------+--------------------+--------------+-------------------+-------------+---------------+--------------+--------------------+-------------------------------+--------+-------------------------+----------------+-----------------+
|13143376000|       206403|         2015-02-14|   League of Legends|      36029255|          riotgames|           -1|        1375163|          True|                  en|               

In [5]:
#remove unnecessary columns
twitch = twitch.drop("broadcasters_created_time").drop("stream_ID").drop("delay_setting").drop("playback_bitrate").drop("source_resolution")
twitch = twitch.drop("broadcaster_name").drop("partner_status")

+-------------+-------------------+--------------------+--------------+---------------+--------------------+-------------------------------+--------+
|current_views|stream_created_time|           game_name|broadcaster_ID|follower_number|broadcaster_language|total_views_of_this_broadcaster|language|
+-------------+-------------------+--------------------+--------------+---------------+--------------------+-------------------------------+--------+
|       206403|         2015-02-14|   League of Legends|      36029255|        1375163|                  en|                      636705939|      en|
|        78525|         2015-02-14|Counter-Strike: G...|      31239503|         238950|                  en|                       26775824|      pl|
|        47007|         2015-02-14|           Minecraft|      15554591|         921635|                  en|                       14392009|      en|
|        31156|         2015-02-14|                H1Z1|      23161357|         690274|             

In [10]:
twitch.createOrReplaceTempView("twitch")
twitch = spark.sql("select current_views, game_name, broadcaster_ID, follower_number, broadcaster_language, total_views_of_this_broadcaster, language, to_date(stream_created_time) as Date from twitch")
twitch.printSchema()
twitch.show()

root
 |-- current_views: string (nullable = true)
 |-- game_name: string (nullable = true)
 |-- broadcaster_ID: string (nullable = true)
 |-- follower_number: string (nullable = true)
 |-- broadcaster_language: string (nullable = true)
 |-- total_views_of_this_broadcaster: string (nullable = true)
 |-- language: string (nullable = true)
 |-- Date: date (nullable = true)

+-------------+--------------------+--------------+---------------+--------------------+-------------------------------+--------+----------+
|current_views|           game_name|broadcaster_ID|follower_number|broadcaster_language|total_views_of_this_broadcaster|language|      Date|
+-------------+--------------------+--------------+---------------+--------------------+-------------------------------+--------+----------+
|       206403|   League of Legends|      36029255|        1375163|                  en|                      636705939|      en|2015-02-14|
|        78525|Counter-Strike: G...|      31239503|         23

In [13]:
twitch_games=spark.sql("SELECT DISTINCT game_name FROM twitch")
twitch_games.show()

+--------------------+
|           game_name|
+--------------------+
|              Dota 2|
|The Elder Scrolls...|
| H1Z1, 7 days to die|
|lets go for that ...|
|                RIFT|
|Legacy of Kain: S...|
|       Ice cold beer|
|          Videogames|
|Pokémon Ruby/Sapp...|
|  Legend of Grimrock|
|Divinity Original...|
|            My Life.|
|           Lets Play|
|           Cleverbot|
|Call of Duty: Bla...|
|            Warcraft|
|              Flower|
|   Bioshock Infinite|
|Super Street Figh...|
|Minecraft: Xbox 3...|
+--------------------+
only showing top 20 rows



In [20]:
twitch.orderby

AnalysisException: '"current_views" is not a numeric column. Aggregation function can only be applied on a numeric column.;'